In [1]:
import acnportal

from copy import deepcopy
import warnings
import pytz
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from acnportal import acnsim
from acnportal import algorithms
from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
import adacharge

In [15]:
def rmit_ev_fleet_network(transformer_cap=30, evse_per_phase=2):
    """ Configurable charging network for level-1 EVSEs connected line to ground
        at 230 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    voltage = 230

    # Define the sets of EVSEs in the Caltech ACN.
    A_ids = ['A-{0}'.format(i) for i in range(evse_per_phase)]
    B_ids = ['B-{0}'.format(i) for i in range(evse_per_phase)]
    C_ids = ['C-{0}'.format(i) for i in range(evse_per_phase)]

    # Add Caltech EVSEs
    for evse_id in A_ids:
        network.register_evse(acnsim.FiniteRatesEVSE(evse_id, [0, 16]), voltage, 0)
    for evse_id in B_ids:
        network.register_evse(acnsim.FiniteRatesEVSE(evse_id, [0, 16]), voltage, 230)
    for evse_id in C_ids:
        network.register_evse(acnsim.FiniteRatesEVSE(evse_id, [0, 16]), voltage, -230)

    # Add Caltech Constraint Set
    I3a = acnsim.Current(A_ids)
    I3b = acnsim.Current(B_ids)
    I3c = acnsim.Current(C_ids)

    # Define intermediate currents
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

In [ ]:
ev_fleet_network = rmit_ev_fleet_network(transformer_cap=30, evse_per_phase=2)

In [ ]:
print(ev_fleet_network.constraint_matrix)
print("-----------------")
print(ev_fleet_network._voltages)
print("-----------------")
print(ev_fleet_network._phase_angles)
print("-----------------")
print(ev_fleet_network.constraint_index)

In [2]:
def rmit_level_2_network(transformer_cap=80, evse_per_phase=10):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 208 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = ChargingNetwork()
    voltage = 400
    evse_type = 'AeroVironment'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(evse_per_phase)]
    BC_ids = ['BC-{0}'.format(i) for i in range(evse_per_phase)]
    CA_ids = ['CA-{0}'.format(i) for i in range(evse_per_phase)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

In [3]:
level2_ev_fleet_network = rmit_level_2_network(transformer_cap=80, evse_per_phase=10)

64.2570281124498
115.94202898550725


In [5]:
print(level2_ev_fleet_network.constraint_matrix)
print("-----------------")
print(level2_ev_fleet_network._voltages)
print("-----------------")
print(level2_ev_fleet_network._phase_angles)
print("-----------------")
print(level2_ev_fleet_network.constraint_index)
print("-----------------")
print(level2_ev_fleet_network.current_charging_rates)

[[ 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.   -1.   -1.   -1.   -1.
  -1.   -1.   -1.   -1.   -1.   -1.  ]
 [-1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.    1.    1.
   1.    1.    1.    1.    1.    1.    1.    1.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   -1.   -1.
  -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.    1.    1.    1.    1.
   1.    1.    1.    1.    1.    1.  ]
 [ 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25
   0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25 -0.5  -0.5  -0.5  -0.5
  -0.5  -0.5  -0.5  -0.5  -0.5  -0.5 ]
 [-0.5  -0.5  -0.5  -0.5  -0.5  -0.5  -0.5  -0.5  -0.5  -0.5   0.25  0.25
   0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25
   0.25  0.25  0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25 -0.5  -0.5
  -0.